## SupraMolSim Analysis workflows focus on exploting its core methods (showcased in general workflows) within parameter sweeps to explore the imaging space!


### In order to do so, we will use the Experiment module, desinged to ease the navigation of SupraMolSim functionallity at any point in the workflow.
The Experiment itself allows you to initialise a demo experiment


In [1]:
from supramolsim.analysis import _plots, sweep
import numpy as np

In [ ]:
structures = ["7R5K", ]
probes=["Mock_linker", ]
modalities = ["STED", "Confocal"]
probe_parameters = sweep.probe_parameters_sweep(labelling_efficiency=[0.5,1,2])
vsample_parameters = sweep.virtual_sample_parameters_sweep(random_orientations=[False,])
acquisition_params = sweep.acquisition_parameters_sweep(exp_time=[0.001, 0.1, 2])
sweep_repetitions = 3

# Create references

In [ ]:
ref_probe_parameters = dict(
    target_info = dict(type="Sequence", value="ELAVGSL"),
    labelling_efficiency = 1,
    distance_to_epitope = 0
)
ref_vsample, ref_params = sweep.generate_global_reference_sample(structure=structures[0], probe="Mock_linker", probe_parameters=ref_probe_parameters)
ref_image, ref_image_pars = sweep.generate_global_reference_modality(reference_vsample=ref_vsample, reference_vsample_params=ref_params)


# sweep over parameters

In [ ]:
myexperiment, vsmpl_output, vsampl_pars = sweep.sweep_vasmples(
    structures=structures,
    probes=probes, 
    probe_parameters=probe_parameters, 
    virtual_samples=vsample_parameters,
    repetitions=sweep_repetitions)

In [ ]:
myexperiment, mod_outputs, mod_params, mod_pixelsizes  = sweep.sweep_modalities_updatemod(
    experiment = myexperiment,
    vsample_outputs = vsmpl_output,
    vsampl_pars = vsampl_pars,
    modalities=modalities,
    modality_acq_prams=acquisition_params
    )

# explore outputs

In [ ]:
import matplotlib.pyplot as plt
param_id = list(mod_params.keys())[3]
repetition = 0
frame = 0

plt.imshow(mod_outputs[param_id][repetition][frame])
print(mod_params[param_id])

# analyse sweep outputs

In [ ]:
measurements, inputs = sweep.analyse_sweep_single_reference(mod_outputs, mod_params, ref_image[0], ref_image_pars)
dframe, combined = sweep.measurements_dataframe(
    measurement_vectors=measurements,
    probe_parameters=probe_parameters,
    mod_acq=acquisition_params,
    mod_names=modalities)

# Plot measurements

In [ ]:
df_categories, titles = sweep.pivot_dataframes_byCategory(combined, "modality_name", "labelling_efficiency", "distance_to_epitope")
_plots.sns_heatmap_pivots(df_categories, titles, annotations=True)